<a href="https://colab.research.google.com/github/rssubramaniyan1/EVA8/blob/main/EVA8_Assignment6_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# **Depthwise Separable Convolution**

The depthwise separable convolution is a combination of depthwise convolution and pointwise convolution.

Depthwise convolution is a type of convolution that applies a single filter to each input channel separately. This means that for an input with N channels and a kernel size of KxK,depthwise convolution applies N kernels to the input.


The pointwise convolution is a 1x1 convolution that applies a single 1x1 to each input channel (i.e., each feature map) separately. This means that for an input with N channels and a filter size of 1x1,pointwise convolution applies N filters of size 1x1 to the input, resulting in N output channels.

These two operations are often combined in a single layer called a depthwise separable convolution.This combines a depthwise convolution with a pointwise convolution to achieve both computational efficiency and strong representational power.


In [ ]:
class depthwise_separable_conv(nn.Module):
    def __init__(self, nin, nout, kernel_size, padding,stride, bias=False):
        super(depthwise_separable_conv, self).__init__()
        self.depthwise = nn.Conv2d(nin, nin, kernel_size=kernel_size, padding=padding, stride=stride, groups=nin, bias=bias)
        self.pointwise = nn.Conv2d(nin, nout, kernel_size=1, bias=bias)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Conv Block 1
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),

            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(0.1)

        )
        # Transition Block 1
        self.pool1 = nn.MaxPool2d(2, 2) # output_size = 16


        # Conv Block 2
        self.convblock3 = nn.Sequential(
            depthwise_separable_conv(64, 32, 3, 1, 1),

            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),

            depthwise_separable_conv(32, 32, 3, 1, 1),

            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),

            depthwise_separable_conv(32, 64, 3, 1, 1),

            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(0.1)
        )

        # Transition Block 2
        self.pool2 = nn.MaxPool2d(2, 2) # output_size = 8

        # Conv Block 3
        self.convblock4 = nn.Sequential(
            depthwise_separable_conv(64, 32, 3, 1, 1),

            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),

            depthwise_separable_conv(32, 32, 3, 1, 1),

            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),

            depthwise_separable_conv(32, 64, 3, 1, 1),

            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(0.1)
        )

        # Transition Block 3
        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=7 )
        ) # output_size = 1

        self.fc1 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.pool1(x)
        x = self.convblock3(x)
        x = self.pool2(x)
        x = self.convblock4(x)
        x = self.gap(x)
        x = x.view(-1, 64)
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)